## First Article Scraping

In [1]:
#make the imports
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#setup browser variable for Chrome, making sure splinter is setup and 
#updated
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\Dell\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [3]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

In [4]:
#visit the url
browser.visit(url)

In [5]:
#after inspecting, get to the info for the latest article with soup
html = browser.html
soup = bs(html, 'html.parser')
content_titles = soup.find_all('div', class_='content_title')
#for some reason it says the first article is Mars Now, though it is
#actually NASA's Mars Fleet Lies Low with Sun Between Earth and Red Planet
#so I'll go deeper into content_titles to get that

In [6]:
#since I used find_all() I got a list back, and can index for the second
#value, which I know is the article I'm look for from glancing at the site
further = content_titles[1]
further

<div class="content_title"><a href="/news/9051/nasas-mars-fleet-lies-low-with-sun-between-earth-and-red-planet/" target="_self">NASA's Mars Fleet Lies Low with Sun Between Earth and Red Planet</a></div>

In [7]:
#now get just the text for the title
news_title = further.text
news_title

"NASA's Mars Fleet Lies Low with Sun Between Earth and Red Planet"

In [8]:
#next go in and get the text for the description of the article
news_p = soup.find('div', class_='article_teaser_body').text
print(news_p)

The missions will continue collecting data about the Red Planet, though engineers back on Earth will stop sending commands to them until mid-October.


## Featured Image Scraping

In [9]:
#visit link in assignment and get the url for the featured image
image_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"

In [10]:
#visit in the notebook
browser.visit(image_url)

In [11]:
#follow same setup steps in previous section for the button
html = browser.html
soup = bs(html, 'html.parser')

In [12]:
#this is from not working from some reason, I will try examining 
#the button itself. It gives mars1 when it should return mars 3

#it worked when restarting the kernel. NASA changed the image while
#I was working, which confused me
image_full = soup.find('img', class_="headerimage fade-in")['src']
image_full

'image/featured/mars3.jpg'

In [13]:
full_image_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/" + image_full

In [14]:
print(full_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg


## On to Mars Facts

In [15]:
# I found this solution to the error code
#urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1125)
# at https://moreless.medium.com/how-to-fix-python-ssl-certificate-verify-failed-97772d9dd14c

import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

In [16]:
mars_facts_df = pd.read_html('https://space-facts.com/mars/')


In [17]:
#this is not a dataframe at the moment, it is a big list, but the
#first item has the values desired for this project
mars_facts_df

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [18]:
mars_facts_df = mars_facts_df[0]

In [19]:
mars_facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [20]:
#name the columns
mars_facts_df.columns=['Statistic', "Value"]

In [21]:
mars_facts_df

,Statistic,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [22]:
html_mars_facts = mars_facts_df.to_html()
html_mars_facts

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Statistic</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC

In [23]:
type(html_mars_facts)

str

## Onto Hemispheres

In [24]:
hemis_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemis_url)

In [93]:
html = browser.html
soup = bs(html, 'html.parser')
links_from_home = []

#using find_all returns a list with repeat values, the only
#first_links = browser.find_by_css("a.product-item h3")
list_h = soup.find_all('a', class_='itemLink product-item')
only_one = []

#get only the full links to follow from the given page:
#https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars

for link in list_h:
    #print(link)
    #print('--------------------')
    if link['href'] not in only_one:
        #print(link['href'])
        only_one.append(link['href'])
        
print(only_one)

part_one = 'https://astrogeology.usgs.gov'
for link in only_one:
    links_from_home.append(part_one + link)
links_from_home

['/search/map/Mars/Viking/cerberus_enhanced', '/search/map/Mars/Viking/schiaparelli_enhanced', '/search/map/Mars/Viking/syrtis_major_enhanced', '/search/map/Mars/Viking/valles_marineris_enhanced']


['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [101]:
#loop through the list of links
hemisphere_image_urls = []

for link in links_from_home:
    #set up a dictionary and re-establish browser for each hemisphere
    #site visited
    hemisphere_dict = {}
    browser.visit(link)
    html = browser.html
    soup = bs(html, 'html.parser')
    
    #get the titles
    test = browser.find_by_css('h2.title').text
    print(test)
    hemisphere_dict['title'] = test
    
    #get the final urls
    image_url = soup.find('img', class_='wide-image')['src']
    image_url = part_one + image_url
    print(image_url)
    hemisphere_dict['img_url'] = image_url
    
  
    hemisphere_image_urls.append(hemisphere_dict)
    
    
hemisphere_image_urls

Cerberus Hemisphere Enhanced
https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
Schiaparelli Hemisphere Enhanced
https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
Syrtis Major Hemisphere Enhanced
https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg
Valles Marineris Hemisphere Enhanced
https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [103]:
browser.quit()